In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)
response=llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-36827?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Lead Software Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc.

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [4]:
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer, ITC',
 'experience': '8+ years hands on experience in technical architecture, design, development & of demand or inventory planning products/applications',
 'skills': ['PL/SQL',
  'Cloud environments (AWS, Azure or GCP)',
  'Integrated Business Planning solutions like O9',
  'Scrum and other Agile processes',
  'Jira, Git/SVN, Jenkins, CI/CD pipelines',
  'Strong techno-functional experience in planning business domain, data domains, planning algorithms and solvers'],
 'description': 'Lead and mentor a team of engineers with a focus to develop solutions for the various products of Demand & Supply Planning capability. Advocate and implement business transformation solutions and best practices. Advocate DevOps practices & mindset and strong culture of innovation.'}

In [6]:
type(json_res)

dict

In [7]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [26]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}]]

In [27]:
job=json_res
job['skills']

['PL/SQL',
 'Cloud environments (AWS, Azure or GCP)',
 'Integrated Business Planning solutions like O9',
 'Scrum and other Agile processes',
 'Jira, Git/SVN, Jenkins, CI/CD pipelines',
 'Strong techno-functional experience in planning business domain, data domains, planning algorithms and solvers']

In [28]:
print(type(job))
print(job)


<class 'dict'>
{'role': 'Lead Software Engineer, ITC', 'experience': '8+ years hands on experience in technical architecture, design, development & of demand or inventory planning products/applications', 'skills': ['PL/SQL', 'Cloud environments (AWS, Azure or GCP)', 'Integrated Business Planning solutions like O9', 'Scrum and other Agile processes', 'Jira, Git/SVN, Jenkins, CI/CD pipelines', 'Strong techno-functional experience in planning business domain, data domains, planning algorithms and solvers'], 'description': 'Lead and mentor a team of engineers with a focus to develop solutions for the various products of Demand & Supply Planning capability. Advocate and implement business transformation solutions and best practices. Advocate DevOps practices & mindset and strong culture of innovation.'}


In [29]:
job

{'role': 'Lead Software Engineer, ITC',
 'experience': '8+ years hands on experience in technical architecture, design, development & of demand or inventory planning products/applications',
 'skills': ['PL/SQL',
  'Cloud environments (AWS, Azure or GCP)',
  'Integrated Business Planning solutions like O9',
  'Scrum and other Agile processes',
  'Jira, Git/SVN, Jenkins, CI/CD pipelines',
  'Strong techno-functional experience in planning business domain, data domains, planning algorithms and solvers'],
 'description': 'Lead and mentor a team of engineers with a focus to develop solutions for the various products of Demand & Supply Planning capability. Advocate and implement business transformation solutions and best practices. Advocate DevOps practices & mindset and strong culture of innovation.'}

In [30]:
job = json_res
job['skills']

['PL/SQL',
 'Cloud environments (AWS, Azure or GCP)',
 'Integrated Business Planning solutions like O9',
 'Scrum and other Agile processes',
 'Jira, Git/SVN, Jenkins, CI/CD pipelines',
 'Strong techno-functional experience in planning business domain, data domains, planning algorithms and solvers']

In [34]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Shashank, a final-year Computer Science student at VIT with strong expertise in Java, Python, JavaScript, and Django. You have internship experience at IRCTC, where you developed a payroll system, and a wide range of project experience in fields like web development, machine learning, and simulation modeling.
        Your task is to write a cold email to the hiring manager, showcasing your technical proficiency while addressing their specific requirements. Mention your projects selectively, but only if they align with the job's required skills. Do not list or stick all projects everywhere.
        
        Available projects and skills to mention:
        
        - Java (75CodeStrong: [75CodeStrong](https://github.com/shashank651156/75CodeStrong))
        - JavaScript (AutoPlayPauseYouTubeVideo-Extension: [AutoPlayPauseYouTubeVideo-Extension](https://github.com/shashank651156/AutoPlayPauseYouTubeVideo-Extension))
        - Python, scikit-learn (Miniature-engine-Spam-Mail-Detection: [Miniature-engine-Spam-Mail-Detection](https://github.com/shashank651156/miniature-engine-Spam-Mail-Detection-))
        - Python (MonteCarloSimulation-usingManufacturingVars: [MonteCarloSimulation-usingManufacturingVars](https://github.com/shashank651156/MonteCarloSimulation-usingManufacturingVars))
        - Django, HTML, CSS, JavaScript (RoomTalk: [RoomTalk](https://github.com/shashank651156/RoomTalk-aDiscordLikeWebApplication))
        
        Use these projects only when they directly fit the skills required in the job description. Focus on how your experience and skills can meet the company's needs while offering concrete examples of your past work when relevant.
        make it like human written.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Lead Software Engineer, ITC Position

Dear Hiring Manager,

I came across the Lead Software Engineer, ITC role at your organization and was impressed by the opportunity to lead a team of engineers in developing innovative solutions for demand and supply planning capabilities. As a final-year Computer Science student at VIT with a strong passion for software development, I am excited to apply for this position and contribute my skills and experience to drive business transformation and innovation.

With a solid foundation in programming languages such as Java, Python, and JavaScript, I am confident in my ability to adapt to new technologies and environments. Although I don't have direct experience with PL/SQL, I am eager to learn and expand my skill set to meet the requirements of the role. My experience with Django, a high-level Python web framework, has taught me the importance of scalable and efficient software design, which I believe would be valuable in dev

In [ ]:
mk